# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos de OpenAI**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

from sklearn.model_selection import train_test_split
from collections import Counter

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
# Incluye las celdas necesarias para tu acceso a la API de OpenAI.



# **Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [84]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

ruta_amazon = '/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 2/Semana 5/Actividad 3/Data/amazon5.txt';
ruta_imdb   = '/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 2/Semana 5/Actividad 3/Data/imdb5.txt';
ruta_yelp   = '/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 2/Semana 5/Actividad 3/Data/yelp5.txt';

amazon_df = pd.read_csv(ruta_amazon, sep="\t", header=None)
imdb_df   = pd.read_csv(ruta_imdb,   sep="\t", header=None)
yelp_df   = pd.read_csv(ruta_yelp,   sep="\t", header=None)

df = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)

# *********** Aquí termina la sección de agregar código *************


In [85]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3000 non-null   object 
 1   1       2000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


In [86]:
# Y veamos sus primeros registros:

df.head()

,0,1
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0


# **Pregunta - 2:**

Realiza el proceso de limpieza. Aplica el preprocesamiento que consideres adecuado.











In [87]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def limpiar_texto(texto):
    # Minusculas
    texto = texto.lower()
    # Tokenizacion
    tokens = word_tokenize(texto, language='english')
    # Caracteres alfabeticos
    tokens = [t for t in tokens if t.isalpha()]
    # Stopwords
    tokens = [t for t in tokens if t not in stop_words]
    # Caracteres repetidos
    tokens = [re.sub(r'(.)\1{2,}', r'\1\1', t) for t in tokens]
    # Lematización
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # Longitud mayor a 1
    tokens = [t for t in tokens if len(t) > 1]

    return tokens

Xclean = df[0].astype(str).apply(limpiar_texto)
Y = df[1]

# *********** Aquí termina la sección de agregar código *************

In [88]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['way', 'plug', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversation', 'lasting', 'problem']
['mic', 'great']


# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [89]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta - 4:**



Construye tu vocabulario a continuación


In [90]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

midiccionario = Counter()

for tokens in x_train:
    midiccionario.update(tokens)

min_freq =10
midicc = {key: freq for key, freq in midiccionario.items() if freq >= min_freq}

# *********** Aquí termina la sección de agregar código *************

In [91]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')

# ******* Inicia la sección de agregar código: ***********

print(len(midicc))

# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
250


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

El vocabulario debe construirse únicamente a partir del conjunto de entrenamiento. Ya que si se incluye datos de validación o prueba en la generación del vocabulario, implicaría que el modelo conoceria de forma anticipada información que se supone debe ser desconocida, lo cual puede comprometer la validez de los resultados.

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [92]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


train_x = []
for ss in x_train:
  train_x.append([w for w in ss if w in midicc])

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in midicc])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in midicc])


# *********** Aquí termina la sección de agregar código *************


In [93]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['much', 'better', 'people', 'like']
['special', 'sucked']
['felt', 'server', 'terrible', 'job']
['call', 'ca', 'steak']
['however']


# **Pregunta - 5:**


#### **Incluye aquí un resumen de las características y diferencias que tiene al menos los tres modelos de OpenAI indicados: "text-embedding-3-small", "text-embedding-3-large" y "text-embedding-ada-002".**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


# **Pregunta - 6:**


#### **Diccionario clave-valor de palabras del diccionario y vectores embebidos.**

In [78]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 7:**



Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [79]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [80]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

NameError: name 'trainEmb' is not defined

# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 9:**



Reporte del mejor modelo con el conjunto de Prueba (Test).


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 10:**

In [ ]:
# Incluye todas las líneas de código y celdas que consideres adecuadas para este ejercicio.


None




# **Pregunta - 11:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

# **Fin de la Actividad de Vectores Embebidos - OpenAI**